## Problem 36
The decimal number,  585=10010010012
  (binary), is palindromic in both bases.

Find the sum of all numbers, less than one million, which are palindromic in base  10
  and base  2
 .

(Please note that the palindromic number, in either base, may not include leading zeros.)

In [1]:
import numpy as np

In [2]:
def check_palindrome(x):
    if type(x) != str:
        x = str(x)
    if x == x[::-1]:
        return True
    return False

def check_palindrome_base2(x):
    base2 = np.base_repr(x, 2)
    return check_palindrome(base2)

def check(x):
    if check_palindrome(x):
        if check_palindrome_base2(x):
            return True
    return False


In [3]:
palindromes = [x for x in range(1000000) if check(x)]

In [4]:
palindromes

[0,
 1,
 3,
 5,
 7,
 9,
 33,
 99,
 313,
 585,
 717,
 7447,
 9009,
 15351,
 32223,
 39993,
 53235,
 53835,
 73737,
 585585]

In [5]:
sum(palindromes)

872187

In [6]:
#the palindrome-first approach
def create_base2_strings(last_string):
    if len(last_string) > 20:
        return []
    else:
        # Add a one:
        next_is_one = last_string + "1"
        futures_with_one = create_base2_strings(next_is_one)

        # Add a zero:
        next_is_zero = last_string + "0"
        futures_with_zero = create_base2_strings(next_is_zero)

        # Combine the results
        return [last_string] + futures_with_one + futures_with_zero

# Start with an empty string
result = create_base2_strings("")

In [7]:
#drop the ones that end with 0
base_strings = [r for r in result if not r.endswith("0") and len(r) > 0]

In [8]:
#all palindromes in the list and the entire set up to len 10 mirrored on itself
base2_palindromes = [x for x in base_strings if check_palindrome(x)] + [x[::-1]+x for x in base_strings if len(x) <= 10]
base2_palindromes = np.unique(base2_palindromes)

In [9]:
dec10 = [int(b, 2) for b in base2_palindromes]

In [10]:
dec10_palindromes = [d for d in dec10 if check_palindrome(d)]

In [11]:
sum(dec10_palindromes)

872187